In [23]:
import urllib
from bs4 import BeautifulSoup
from os import path
from urllib.parse import urljoin
import re
import csv

In [24]:
BASE = ''
CAT_BASE = 'jp/news/products/'
CATEGORY = []
TARGET = '/jp/news/story/'

OUT_PUT = 'test.tsv'

In [8]:
url_dict = {}
for cat_idx, cat in enumerate(CATEGORY):
    # 各カテゴリ内の'jp/news/products/'から始まるリンクのURLを取得
    html = urllib.request.urlopen(urljoin(BASE, CAT_BASE+cat+'.html'))
    soup = BeautifulSoup(html)
    target_tags = soup.select('a[href^="'+TARGET+'"]')

    # 取得したリンクを辞書に代入する
    for tag in target_tags:
        target_url = tag.attrs['href']
        if target_url in url_dict:
            url_dict[target_url].append(cat_idx)
        else:
            url_dict[target_url] = [cat_idx]

In [18]:
print(len(url_dict))
print(target_url)

783
/jp/news/story/1109275109.html


In [43]:
with open(OUT_PUT, 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    
    for url, categorys in url_dict.items():
        # 各URLから記事に関するタグを取得
        html = urllib.request.urlopen(urljoin(BASE, url))
        soup = BeautifulSoup(html)
        news_block_tags = soup.findAll('section', id=re.compile('news_block*'))

        # タグの中からテキストのみを抽出
        sentence = ''
        for i, tag in enumerate(news_block_tags):
            text = tag.get_text()

            # 不要な文字列を削除
            text = re.sub('MHI_MODULES.renderNewsSummary(".+")', '', text)
            text = re.sub('<.+/>', '', text)
            text = re.sub('[\r,\n]', ' ', text)
            text = re.sub(' +', ' ', text)
            sentence += ' '+text if i > 0 else text
        sentence =  re.sub('^ *', '', sentence)
        sentence =  re.sub(' *$', '', sentence)

        # TODO カテゴリの重複処理が必要
        writer.writerow([str(categorys[0]), sentence])